In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
response = requests.get("https://www.fca.org.uk/news/search-results?category=warnings&start=1", headers={"Accept-Language":"en-US"})
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
# def company_name(soup):
#   companies = []
#   # try:
#   #   for company in soup.find_all('li', class_='search-item first'):
#   #     names = company.find("h4").find("a").string.strip()
#   #     companies.append(names)
#   # except:
#   #   print('top error')
#   try:
#     for company in soup.find_all("li", class_='search-item'):
#       names = company.find("h4").find("a").string.strip()
#       companies.append(names)
#   except:
#     print('body error')
  
#   # try:
#   #   for company in soup.find_all('li', class_='search-item last'):
#   #     names = company.find("h4").find("a").string.strip()
#   #     companies.append(names)
#   # except:
#   #   print('last error')
  
  #return companies 

In [ ]:
import time
def get_all_company_names(no_reported_companies):
    all_company_names = []
    
    for i in list(range(1, no_reported_companies, 10)): 
        print(f'scraping page {i}')
        params = {'category': 'warnings',
                 'start': i}
        url = 'https://www.fca.org.uk/news/search-results'
        response = requests.get(url, params=params, headers={"Accept-Language":"en-US"})
        
        if response.status_code != 200:
            print(f'response error: {response.status_code} on page {i}')
            break
        
        soup = BeautifulSoup(response.content, "html.parser")
        
        try:
            for company in soup.find_all('li', class_='search-item'):
                names = company.find('h4').find('a').string.strip()
                print(names)
                all_company_names.append(names)
        except:
            print(f'error on page {i}')
        
        # Sleep
        time.sleep(3)

    return all_company_names

In [ ]:
#This is the total number of company names listed on the FCA watch list 

search_results = int(soup.find('p', class_='search-results__count col-xs-12').find('strong').find_next('strong').find_next('strong').string)

search_results

10315

In [ ]:
every_company_name = get_all_company_names(search_results)

In [ ]:
#every_company_name = pd.DataFrame(np.array(all_company_names))
every_company_name.drop_duplicates(inplace=True)

In [ ]:
list(every_company_name[0])

In [ ]:
every_company_name.to_csv('data.csv')
!cp data.csv "drive/My Drive/"

In [ ]:
import re
clone_pattern = r'[(C|(c]lone\s|.+?[)]'
clones = []
for company in list(every_company_name[0]):
    if re.match(clone_pattern, company):
      fixed = company.split(' (')[0]
      clones.append(fixed)

In [ ]:
len(clones)

3134

In [ ]:
clone_url_options = ['-clone-fca-authorised-firm', '-clone-previously-registered-firm', '-clone-fca-authorised-schedule-5-firm', '-clone-fca-registered-firm', '-clone-appointed-representative', '-fca-exempt-firm', '-registered-psd-agent', '-former-psd-agent', ]
clone_final_urls = []

def check_options(clone):
    new_company = clone.replace(' ', '-').replace('.', '').replace('(', '').replace(')','').replace('://','-').replace('-of','').replace('/', '-').lower()
    new_url=f"https://www.fca.org.uk/news/warnings/{new_company}"
    
    for i in clone_url_options:
      check_url = new_url + i
      print(check_url)
      try:
        response = requests.get(check_url)
        if response.status_code == 200:
          clone_final_urls.append(check_url)
      except:
        continue





In [ ]:
for clone in clones:
  check_options(clone)

In [ ]:
clone_final_urls

['https://www.fca.org.uk/news/warnings/maryamfx-clone-fca-authorised-firm',
 'https://www.fca.org.uk/news/warnings/wwwcryptosaurcouk-clone-previously-registered-firm',
 'https://www.fca.org.uk/news/warnings/https-wwwwintonfundmanagementie-clone-fca-authorised-schedule-5-firm']

In [ ]:
#companies = soup.find('p', class_='search-results__count col-xs-12'  )

In [ ]:
#RETRIEVING ALL COMPANY NAMES FROM SCAM LIST
# def get_all_company_names(start, end):
#   all_companies = []
#   for i in range(start, end, 10): 
#     print(f'scraping page {i}')
#     response = requests.get(f"https://www.fca.org.uk/news/search-results?category=warnings&start={i}", headers={"Accept-Language":"en-US"})
#     soup = BeautifulSoup(response.content, "html.parser")
#     # print(soup)
#     all_companies += company_name(soup)

#   return all_companies


In [ ]:
# def list_func(all_comp):
#   l=[]
#   e = []
#   for i, page in enumerate(all_comp):
#     if page:
#       l += page
#     else:
#       print(f'page {i} is EMPTY')
#       e += page
#   return l, e

In [ ]:
#list_companies, empty_pages = list_func(all_company_names)

In [ ]:
#non_clone_company_names = every_company_name - clones

In [ ]:
def get_urls(all_company_names):

  list_url=[]

  for company in all_company_names:

    #print(company)
    new_company=company.replace(' ', '-').replace('.', '').replace('(', '').replace(')','').replace('://','-').replace('-of','').replace('/', '-').lower()
    new_url=f"https://www.fca.org.uk/news/warnings/{new_company}"
    
    list_url.append(new_url)

  #pd.DataFrame(np.array(list_url))
  return list_url

In [ ]:
list_companies

In [ ]:
get_urls(list_companies[0:5])

['https://www.fca.org.uk/news/warnings/cryptoforextrading50',
 'https://www.fca.org.uk/news/warnings/cryptodigital-currency',
 'https://www.fca.org.uk/news/warnings/highstarinvestcom',
 'https://www.fca.org.uk/news/warnings/crypto-global-trades',
 'https://www.fca.org.uk/news/warnings/forex-crypto-payout']

In [ ]:
some_urls=get_urls(list_companies[0:20])
some_urls


In [ ]:
from google.colab import files
def scam_url(some_urls, list_companies):
  df={}
  for i, url in enumerate(some_urls):
    print(i+1)
    response = BeautifulSoup(requests.get(url).content, "html.parser")
    for tag in response.find_all('p'):
      text = tag.text
      website_tag = "Website: "
      if text.startswith(website_tag):
        website = text[len(website_tag):]
        if i in df:
          df[i]['website'].append(website)
        else:
          df[i]={'company name': list_companies[i], 'fca website':url, 'website': [website]}
        print(website)
        output_df = pd.DataFrame.from_dict(df, orient='index')
        if (i+1)%5==0:
          output_df.to_csv(f'/content/data_{i+1}.csv')
          files.download(f'/content/data_{i+1}.csv')
  return output_df


In [ ]:
scam_url(some_urls, list_companies)

In [ ]:
some_links_0_100 = scam_url(some_urls)

some_links_0_100

In [ ]:
for i in some_urls:
  response = BeautifulSoup(requests.get(i).content, "html.parser")
  for tag in response.find_all('p'):
    text = tag.text
    website_tag = "Website: "
    if text.startswith(website_tag):
      website = text[len(website_tag):]
      print(website)

